### Chapter 4 Exercises

In [2]:
import polars as pl

pl.Config.set_fmt_str_lengths(50)

polars.config.Config

1.	Use Polars to read the Parquet file nypd_crimes_from_2018 from the dataset folder. 
Filter the dataset for the dates of 31st December 2019 and 1st January 2020.
We will use this filtered dataset for the rest of the exercises

In [5]:
nypd = pl.read_parquet('../datasets/nypd_crimes_from_2018.parquet')

nypd = nypd.with_columns(
    pl.col('complaint_datetime').str.to_datetime("%Y-%m-%d %H:%M:%S")
)

In [50]:
from datetime import datetime

nypd_filtered = nypd.filter(
    pl.col('complaint_datetime')
    .is_between(datetime(2019, 12, 31), datetime(2020, 1, 2))
)

2.	Take the dataset from question 1, group it dataset by crime_category, and add 2 list columns: one with a list of precincts, and another with a list of boroughs. Do not worry about duplicated values in the lists.

In [51]:
nypd_grouped = nypd_filtered.groupby('crime_category').agg(
    'precinct', 'borough'
)

nypd_grouped

crime_category,precinct,borough
str,list[i64],list[str]
"""VIOLATION""","[20, 42, … 73]","[""MANHATTAN"", ""BRONX"", … ""BROOKLYN""]"
"""MISDEMEANOR""","[19, 48, … 44]","[""MANHATTAN"", ""BRONX"", … ""BRONX""]"
"""FELONY""","[105, 114, … 72]","[""QUEENS"", ""QUEENS"", … ""BROOKLYN""]"


3.	Start with the grouped dataframe of question 2, then add the following columns:

- 'borough_unique' with the list of boroughs without duplicated values
- 'precinct_count' with the count of precincts (you need to remove duplicated values first)
- 'top_borough' with the borough having the most crimes (hint: check out the 'mode' method)

In [52]:
nypd_grouped.with_columns(
    borough_unique = pl.col('borough').list.unique(),
    precinct_count = pl.col('precinct').list.unique().list.lengths(),
    top_borough = pl.col('borough').list.eval(pl.element().mode())
)

crime_category,precinct,borough,borough_unique,precinct_count,top_borough
str,list[i64],list[str],list[str],u32,list[str]
"""VIOLATION""","[20, 42, … 73]","[""MANHATTAN"", ""BRONX"", … ""BROOKLYN""]","[""BROOKLYN"", ""QUEENS"", … ""BRONX""]",74,"[""QUEENS""]"
"""MISDEMEANOR""","[19, 48, … 44]","[""MANHATTAN"", ""BRONX"", … ""BRONX""]","[""BROOKLYN"", ""BRONX"", … ""MANHATTAN""]",77,"[""BROOKLYN""]"
"""FELONY""","[105, 114, … 72]","[""QUEENS"", ""QUEENS"", … ""BROOKLYN""]","[""QUEENS"", ""BRONX"", … ""MANHATTAN""]",76,"[""BROOKLYN""]"


4.	Go back to the filtered dataframe of question 1, then filter it to keep only rows where crime_description contains the word 'COMMERCIAL'

In [53]:
nypd_filtered.filter(
    pl.col('crime_description').str.contains('COMMERCIAL')
)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
datetime[μs],str,i64,str,str,str
2019-12-31 05:45:00,"""102079547""",18,"""MANHATTAN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""
2020-01-01 02:30:00,"""268949120""",114,"""QUEENS""","""FELONY""","""BURGLARY,COMMERCIAL,NIGHT"""
2020-01-01 08:35:00,"""938871036""",13,"""MANHATTAN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""
2019-12-31 15:15:00,"""207059189""",6,"""MANHATTAN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""
2020-01-01 06:30:00,"""526074457""",109,"""QUEENS""","""FELONY""","""ROBBERY,COMMERCIAL UNCLASSIFIED"""
2020-01-01 02:54:00,"""430866151""",83,"""BROOKLYN""","""FELONY""","""BURGLARY,COMMERCIAL,NIGHT"""
2019-12-31 12:00:00,"""980287441""",60,"""BROOKLYN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""
2020-01-01 17:50:00,"""770537209""",18,"""MANHATTAN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""
2020-01-01 12:30:00,"""893931305""",84,"""BROOKLYN""","""FELONY""","""BURGLARY,COMMERCIAL,DAY"""


5.	Use the filtered dataframe of question 1, group it by hour using truncate, sort it, then add the following columns

- 'crime_count' with the count of crimes in the hour
- 'crime_count_one_hour_earlier' with the count of crimes one hour earlier
- 'crime_count_24_hours_earlier' with the count of crimes 24 hours earlier
- 'crime_count_difference_one_hour' with the change in the count of crimes vs one hour earlier

Assume that the dataset has data for every hour

In [63]:
(
    nypd_filtered.groupby(
        pl.col('complaint_datetime').dt.truncate('1h'))
    .count()
    .sort('complaint_datetime')
    .rename({'count': 'crime_count'})
    .with_columns(
        crime_count_one_hour_earlier = pl.col('crime_count').shift(1),
        crime_count_24_hours_earlier = pl.col('crime_count').shift(24),
        crime_count_difference_one_hour = pl.col('crime_count').diff(1)
    )
)

complaint_datetime,crime_count,crime_count_one_hour_earlier,crime_count_24_hours_earlier,crime_count_difference_one_hour
datetime[μs],u32,u32,u32,i64
2019-12-31 00:00:00,50,null,null,null
2019-12-31 01:00:00,38,50,null,-12
2019-12-31 02:00:00,18,38,null,-20
2019-12-31 03:00:00,18,18,null,0
2019-12-31 04:00:00,12,18,null,-6
2019-12-31 05:00:00,16,12,null,4
2019-12-31 06:00:00,12,16,null,-4
2019-12-31 07:00:00,24,12,null,12
2019-12-31 08:00:00,26,24,null,2


6.	Use the filtered dataframe of question 1, group it by hour using truncate, sort it, then add the following columns

- 'crime_count' with the count of crimes in the hour
- 'cumulative_crime_count' with the cumulative count of crimes
- 'cumulative_average_crime' with the cumulative average of the crime count
- 'cumulative_median_crime' with the cumulative median of the crime count

Assume that the dataset has data for every hour

In [65]:
(
    nypd_filtered.groupby(
        pl.col('complaint_datetime').dt.truncate('1h'))
    .count()
    .sort('complaint_datetime')
    .rename({'count': 'crime_count'})
    .with_columns(
        cumulative_crime_count = pl.col('crime_count').cumsum(),
        cumulative_average_crime = pl.col('crime_count').cumulative_eval(pl.element().mean()),
        cumulative_median_crime = pl.col('crime_count').cumulative_eval(pl.element().median())
    )
)

complaint_datetime,crime_count,cumulative_crime_count,cumulative_average_crime,cumulative_median_crime
datetime[μs],u32,u32,f64,f64
2019-12-31 00:00:00,50,50,50.0,50.0
2019-12-31 01:00:00,38,88,44.0,44.0
2019-12-31 02:00:00,18,106,35.333333,38.0
2019-12-31 03:00:00,18,124,31.0,28.0
2019-12-31 04:00:00,12,136,27.2,18.0
2019-12-31 05:00:00,16,152,25.333333,18.0
2019-12-31 06:00:00,12,164,23.428571,18.0
2019-12-31 07:00:00,24,188,23.5,18.0
2019-12-31 08:00:00,26,214,23.777778,18.0


7.	Use the filtered dataframe of question 1, group it by hour using truncate, sort it, then add the following columns

- 'crime_count' with the number of crimes in the hour
- 'rolling_5hours_crime_count' with the number of crimes for the past 5 hours
- 'rolling_5hours_average_crime' with the average number of crimes of the past 5 hours
- 'rolling_5hours_median_crime' with the median number of crimes of the past 5 hours
- 'crimes_past_5hours' with a list of the number of crimes for the past 5 hours

Assume that the dataset has data for every hour

In [76]:
(
    nypd_filtered.groupby(
        pl.col('complaint_datetime').dt.truncate('1h'))
    .count()
    .sort('complaint_datetime')
    .rename({'count': 'crime_count'})
    .groupby_rolling(
        index_column = 'complaint_datetime',
        period= '5h')
    .agg(
        rolling_5hours_crime_count = pl.col('crime_count').sum(),
        rolling_5hours_average_crime = pl.col('crime_count').mean(),
        rolling_5hours_median_crime = pl.col('crime_count').median(),
        crimes_past_5hours = pl.col('crime_count')
    )
)

complaint_datetime,rolling_5hours_crime_count,rolling_5hours_average_crime,rolling_5hours_median_crime,crimes_past_5hours
datetime[μs],u32,f64,f64,list[u32]
2019-12-31 00:00:00,50,50.0,50.0,[50]
2019-12-31 01:00:00,88,44.0,44.0,"[50, 38]"
2019-12-31 02:00:00,106,35.333333,38.0,"[50, 38, 18]"
2019-12-31 03:00:00,124,31.0,28.0,"[50, 38, … 18]"
2019-12-31 04:00:00,136,27.2,18.0,"[50, 38, … 12]"
2019-12-31 05:00:00,102,20.4,18.0,"[38, 18, … 16]"
2019-12-31 06:00:00,76,15.2,16.0,"[18, 18, … 12]"
2019-12-31 07:00:00,82,16.4,16.0,"[18, 12, … 24]"
2019-12-31 08:00:00,90,18.0,16.0,"[12, 16, … 26]"
